In [1]:
from kitti_detection import config
from kitti_detection.dataset import DataSample, class_names, load_train_val_test_dataset
from kitti_detection.utils import display_samples_h
from kitti_detection import box_utils

import torch
import torchvision
from torch import nn, optim, Tensor
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision.transforms import v2
from torchvision.tv_tensors import BoundingBoxes

from scipy.optimize import linear_sum_assignment

In [2]:
WIDTH_FACTOR, HEIGHT_FACTOR = 1/370, 1/370

In [3]:
class NormalizeBoundingBoxes(v2.Transform):
    def __init__(self):
        super().__init__()

    def __call__(self, img, target):
        target['boxes'] *= torch.tensor([WIDTH_FACTOR, HEIGHT_FACTOR, WIDTH_FACTOR, HEIGHT_FACTOR])
        return img, target

In [4]:
transforms = v2.Compose([
    v2.RandomCrop(size=(370, 370)),
    v2.SanitizeBoundingBoxes(),
    v2.ToDtype(torch.float32),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.ConvertBoundingBoxFormat(format='cxcywh'),
    NormalizeBoundingBoxes(),
])

In [5]:
class_names = class_names[:-1]

In [6]:
n_classes = len(class_names)

In [7]:
def p(t):
    print(t)
    print(t.size())
    print()

In [8]:
def get_1d_pos_encoding(l, dim):
    return torch.cat([
        torch.stack([
            torch.linspace(0, 10000**(2*i/dim), steps=l).sin(),
            torch.linspace(0, 10000**(2*i/dim), steps=l).cos()
        ], dim=1)
        for i in range(dim // 2)
    ], dim=1)

def create_pos_encoding(h, w, dim):
    col_embed = get_1d_pos_encoding(w, dim // 2).repeat(h, 1, 1)
    row_embed = get_1d_pos_encoding(h, dim // 2).unsqueeze(1).repeat(1, w, 1)
    
    return torch.cat((col_embed, row_embed), dim=-1)

In [9]:
class DETR(nn.Module):

    def __init__(self, dim_embed=256):
        super().__init__()
        self.backbone = self._backbone()
        self.conv = nn.Conv2d(512, dim_embed, kernel_size=1)

        self.register_buffer('pos_embedding', create_pos_encoding(12, 12, dim_embed)) # (12, 12, 256)
        self.register_buffer('query_pos_embedding', get_1d_pos_encoding(20, dim_embed))

        self.transformer = nn.Transformer(dim_embed, nhead=8, num_encoder_layers=4, num_decoder_layers=4, batch_first=True)

        self.linear_class = nn.Linear(dim_embed, n_classes + 1)
        self.linear_bbox = nn.Linear(dim_embed, 4)

    def _backbone(self) -> nn.Module:
        backbone = torchvision.models.resnet18(weights=torchvision.models.ResNet18_Weights.IMAGENET1K_V1)
        del backbone.fc
        del backbone.avgpool

        def _forward(bb: torchvision.models.ResNet, x):
            x = bb.conv1(x)
            x = bb.bn1(x)
            x = bb.relu(x)
            x = bb.maxpool(x)

            x = bb.layer1(x)
            x = bb.layer2(x)
            x = bb.layer3(x)
            x = bb.layer4(x)
            return x

        backbone.forward = lambda x: _forward(backbone, x)
        # TODO BatchNorm freeze
        return backbone

    def forward(self, input):
        batch_size = input.shape[0]

        x = self.backbone(input) # (4, 512, 12, 12)
        x = self.conv(x) # (4, 256, 12, 12)

        x = x.permute(0, 2, 3, 1) # (4, 12, 12, 256)
        x = x + self.pos_embedding
        x = x.flatten(1, 2) # (4, 144, 256)

        q = self.query_pos_embedding.repeat(batch_size, 1, 1) # (4, 20, 256)
        q = self.transformer(x, q)

        return {
            'pred_logits': self.linear_class(q),
            'pred_boxes': torch.sigmoid(self.linear_bbox(q))
        }


In [10]:
def _collate(samples):
    imgs = tuple( img for img, _ in samples )
    targets = tuple( target for _, target in samples )
    return torch.stack(imgs), tuple(targets)

In [11]:
@torch.no_grad
def match_indices(outputs: torch.Tensor, targets: torch.Tensor, cost_bbox_factor=0.3, cost_class_factor=0.3, cost_giou_factor=0.3):
    """ Performs the matching

    Params:
        outputs: This is a dict that contains at least these entries:
                "pred_logits": Tensor of dim [batch_size, num_queries, num_classes] with the classification logits
                "pred_boxes": Tensor of dim [batch_size, num_queries, 4] with the predicted box coordinates

        targets: This is a list of targets (len(targets) = batch_size), where each target is a dict containing:
                "labels": Tensor of dim [num_target_boxes] (where num_target_boxes is the number of ground-truth
                        objects in the target) containing the class labels
                "boxes": Tensor of dim [num_target_boxes, 4] containing the target box coordinates

    Returns:
        A list of size batch_size, containing tuples of (index_i, index_j) where:
            - index_i is the indices of the selected predictions (in order)
            - index_j is the indices of the corresponding selected targets (in order)
        For each batch element, it holds:
            len(index_i) = len(index_j) = min(num_queries, num_target_boxes)
    """
    bs, num_queries = outputs["pred_logits"].shape[:2]

    # We flatten to compute the cost matrices in a batch
    out_prob = outputs["pred_logits"].flatten(0, 1).softmax(-1)  # [batch_size * num_queries, num_classes]
    out_bbox = outputs["pred_boxes"].flatten(0, 1)  # [batch_size * num_queries, 4]

    # Also concat the target labels and boxes
    tgt_ids = torch.cat([v["labels"] for v in targets])
    tgt_bbox = torch.cat([v["boxes"] for v in targets])

    # Compute the classification cost. Contrary to the loss, we don't use the NLL,
    # but approximate it in 1 - proba[target class].
    # The 1 is a constant that doesn't change the matching, it can be ommitted.
    cost_class = -out_prob[:, tgt_ids]

    # Compute the L1 cost between boxes
    cost_bbox = torch.cdist(out_bbox, tgt_bbox, p=1)

    # Compute the giou cost betwen boxes
    cost_giou = -box_utils.generalized_box_iou(box_utils.box_cxcywh_to_xyxy(out_bbox), box_utils.box_cxcywh_to_xyxy(tgt_bbox))

    # Final cost matrix
    C = cost_bbox_factor * cost_bbox + cost_class_factor * cost_class + cost_giou_factor * cost_giou
    C = C.view(bs, num_queries, -1).cpu()

    sizes = [len(v["boxes"]) for v in targets]
    indices = [linear_sum_assignment(c[i].detach()) for i, c in enumerate(C.split(sizes, -1))]
    return [(torch.as_tensor(i, dtype=torch.int64), torch.as_tensor(j, dtype=torch.int64)) for i, j in indices]

In [12]:
class HungarianLoss(nn.Module):
    """ This class computes the loss for DETR.
    The process happens in two steps:
        1) we compute hungarian assignment between ground truth boxes and the outputs of the model
        2) we supervise each pair of matched ground-truth / prediction (supervise class and box)
    """
    def __init__(self, num_classes, class_weights, empty_weight, loss_weight_dict):
        super().__init__()
        self.num_classes = num_classes
        class_weights = torch.tensor(class_weights)
        class_weights = torch.cat([class_weights, torch.tensor([empty_weight])])
        self.loss_weight_dict = loss_weight_dict
        self.register_buffer('class_weights', class_weights)

    def get_loss_labels(self, outputs, targets, indices):
        """Classification loss (NLL)
        targets dicts must contain the key "labels" containing a tensor of dim [nb_target_boxes]
        """
        src_logits = outputs['pred_logits']

        idx = self._get_src_permutation_idx(indices)
        target_classes_o = torch.cat([t["labels"][J] for t, (_, J) in zip(targets, indices)])
        target_classes = torch.full(src_logits.shape[:2], self.num_classes,
                                    dtype=torch.int64, device=src_logits.device)
        target_classes[idx] = target_classes_o

        loss = F.cross_entropy(src_logits.transpose(1, 2), target_classes, self.class_weights.to(src_logits.device))
        return loss

    def get_loss_boxes(self, outputs, targets, indices, num_boxes):
        """Compute the losses related to the bounding boxes, the L1 regression loss and the GIoU loss
           targets dicts must contain the key "boxes" containing a tensor of dim [nb_target_boxes, 4]
           The target boxes are expected in format (center_x, center_y, w, h), normalized by the image size.
        """
        idx = self._get_src_permutation_idx(indices)
        src_boxes = outputs['pred_boxes'][idx]
        target_boxes = torch.cat([t['boxes'][i] for t, (_, i) in zip(targets, indices)], dim=0)

        loss_bbox = F.l1_loss(src_boxes, target_boxes, reduction='none')
        loss_bbox = loss_bbox.sum() / num_boxes

        loss_giou = 1 - torch.diag(box_utils.generalized_box_iou(
            box_utils.box_cxcywh_to_xyxy(src_boxes),
            box_utils.box_cxcywh_to_xyxy(target_boxes)))
        loss_giou = loss_giou.sum() / num_boxes
        
        return loss_bbox, loss_giou

    def _get_src_permutation_idx(self, indices):
        # permute predictions following indices
        batch_idx = torch.cat([torch.full_like(src, i) for i, (src, _) in enumerate(indices)])
        src_idx = torch.cat([src for (src, _) in indices])
        return batch_idx, src_idx

    def forward(self, outputs, targets):
        indices = match_indices(outputs, targets)

        num_boxes = sum(len(t["labels"]) for t in targets)
        num_boxes = torch.as_tensor([num_boxes], dtype=torch.float, device=outputs['pred_logits'].device)

        loss_labels = self.get_loss_labels(outputs, targets, indices)
        if num_boxes > 0:
            loss_bbox, loss_giou = self.get_loss_boxes(outputs, targets, indices, num_boxes)
        else:
            loss_bbox = loss_giou = 0

        losses = {
            'loss_labels': loss_labels,
            'loss_bbox': loss_bbox,
            'loss_giou': loss_giou,
        }

        loss_sum = sum([ weight * losses[key] for key, weight in self.loss_weight_dict.items() ])
        return loss_sum, losses


In [13]:
train_dataset, valid_dataset, test_dataset = load_train_val_test_dataset()

train_dataset.transform = transforms
valid_dataset.transform = transforms

In [14]:
BATCH_SIZE = 16

In [15]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=_collate)
valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, collate_fn=_collate)

In [16]:
model = DETR()

In [17]:
def targets_to_device(targets, device):
    return tuple({k: v.to(device) for k, v in target.items()} for target in targets)

In [18]:
def train_model(model: nn.Module,
                device: str,
                train_loader: DataLoader,
                valid_loader: DataLoader,
                batch_size: int,
                loss_fn: nn.Module,
                optimizer: optim.Optimizer,
                epochs: int,
                scheduler: optim.lr_scheduler.LRScheduler = None,
                early_stopping_patience: int = None
                ):

    model = model.to(device)
    num_train_samples = len(train_loader.dataset)
    best_loss = float('inf')
    patience = early_stopping_patience

    for epoch in range(epochs):
        # train
        num_batches = len(train_loader)
        model = model.train()
        train_loss = 0.0

        for batch, (X, y) in enumerate(train_loader):
            #print(f'[{epoch + 1:>2}/{epochs}]:', end='\r')
            X = X.to(device)
            y = targets_to_device(y, device)
            
            pred = model(X)
            loss, loss_dict = loss_fn(pred, y)

            # Backpropagation
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            loss, current = loss.item(), batch * batch_size + len(X)
            train_loss += loss
            print(f'[{epoch + 1:>2}/{epochs}]: train_loss={loss:>5f}  [{current:>5d}/{num_train_samples:>5d}]')
        train_loss /= num_batches

        if scheduler:
            scheduler.step()

        # validate
        model = model.eval()
        n_samples = 0
        num_batches = len(valid_loader)
        val_loss = 0

        with torch.no_grad():
            for X, y in valid_loader:
                X = X.to(device)
                y = targets_to_device(y, device)
                pred = model(X)
                loss, _  = loss_fn(pred, y)
                val_loss += loss.item()
                n_samples += len(X)

        val_loss /= num_batches
        print(f'[{epoch + 1:>2}/{epochs}]: {train_loss=:5f} {val_loss=:5.5f}')
        
        patience -= 1

        if val_loss < best_loss:
            best_loss = val_loss
            torch.save(model.state_dict(), config.CURRENT_MODEL_PATH)
            patience = early_stopping_patience
        if patience == 0:
            break

    model.load_state_dict(torch.load(config.CURRENT_MODEL_PATH))
    print(f'\nbest model:')
    print(f'\tval_loss={best_loss:2f}')


In [19]:
class_weights=[1.0 for _ in range(8)]
loss_weight_dict = {'loss_labels': 0.33, 'loss_bbox': 0.33, 'loss_giou': 0.33}
loss_fn = HungarianLoss(num_classes=len(class_names), class_weights=class_weights, empty_weight=0.2, loss_weight_dict=loss_weight_dict)

train_model(
    model,
    device='cuda:0',
    train_loader=train_loader,
    valid_loader=valid_loader,
    batch_size=BATCH_SIZE,
    loss_fn=loss_fn,
    optimizer=optim.AdamW(model.parameters(), lr=0.01, weight_decay=0.001),
    epochs=2,
    early_stopping_patience=10,
)

[ 1/2]: train_loss=1.467735  [   16/ 5237]
[ 1/2]: train_loss=1.642001  [   32/ 5237]
[ 1/2]: train_loss=1.448709  [   48/ 5237]
[ 1/2]: train_loss=1.375795  [   64/ 5237]
[ 1/2]: train_loss=1.402847  [   80/ 5237]
[ 1/2]: train_loss=1.237998  [   96/ 5237]
[ 1/2]: train_loss=0.971650  [  112/ 5237]
[ 1/2]: train_loss=1.066948  [  128/ 5237]
[ 1/2]: train_loss=1.060470  [  144/ 5237]
[ 1/2]: train_loss=0.921089  [  160/ 5237]
[ 1/2]: train_loss=0.946602  [  176/ 5237]
[ 1/2]: train_loss=0.905962  [  192/ 5237]
[ 1/2]: train_loss=1.085009  [  208/ 5237]
[ 1/2]: train_loss=0.930899  [  224/ 5237]
[ 1/2]: train_loss=0.979612  [  240/ 5237]
[ 1/2]: train_loss=1.068821  [  256/ 5237]
[ 1/2]: train_loss=0.973756  [  272/ 5237]
[ 1/2]: train_loss=0.992349  [  288/ 5237]
[ 1/2]: train_loss=0.993598  [  304/ 5237]
[ 1/2]: train_loss=1.083637  [  320/ 5237]
[ 1/2]: train_loss=1.039406  [  336/ 5237]
[ 1/2]: train_loss=0.912185  [  352/ 5237]
[ 1/2]: train_loss=0.912870  [  368/ 5237]
[ 1/2]: tra